In [1]:
import sys; sys.path.append(_dh[0].split("knowknow")[0])
from knowknow import *

database_name = 'sociology-jstor'

In [2]:
cysum = load_variable("%s.cysum" % database_name)
cits = get_cnt("%s.doc" % database_name, ['c.fy'])

Loaded keys: dict_keys(['c.fy'])
Available keys: ['fy', 'c', 'fj', 'fj.fy', 'c.fj', 'c.fy', 'a.c', 'a.fj.fy', 'a', 'c.c', 'c.t', 't', 'fy.t', 'fj.t']


In [3]:
list(cysum)[:5]

['Szymanski (1976)',
 'McGowan and Smith (1978)',
 'Delacroix and Ragin (1981)',
 'Bradshaw and Wahl (1991)',
 'Stoneman (1975)']

In [4]:
toptops = []

print("%s total entries" % len(cysum))


# ranges loop from 1940-1950 to 1980-1990, in 1-year increments
for RANGE_START, RANGE_END in zip( 
    range(1940,1980+1,5),
    range(1950,1990+1,5),
):
    
    # create a copy of cysum
    cysum_filt = [x for x in cysum.values()]

    count_in_range = defaultdict(int)
    for (c,y),count in cits['c.fy'].items():
        if RANGE_END >= y >= RANGE_START:
            count_in_range[c] += count

    for c in cysum_filt:
        if c['name'] in count_in_range:
            c['count_in_range'] = count_in_range[c['name']]
            
    #cysum_filt = [x for x in cysum_filt if x['count_in_range']>=5]
    #print("%s of them have at least 5 citations in range" % len(cysum_filt))

    alldf = pd.DataFrame.from_records([
        cysum[c] for c in count_in_range
        if c in cysum
    ])
    print(alldf.shape)
    
    q99 = alldf.count_in_range.quantile(0.99)
    toptop = alldf[alldf.count_in_range>=q99]
    
    
    print("%s /%s in the top 1%% in %s,%s" % (
        toptop.shape[0],
        alldf.shape[0],
        RANGE_START, RANGE_END
    ))
    
    toptop = pd.DataFrame.from_records(toptop)
    toptops.append(toptop)

141608 total entries
(168, 15)
2 /168 in the top 1% in 1940,1950
(214, 15)
3 /214 in the top 1% in 1945,1955
(332, 15)
6 /332 in the top 1% in 1950,1960
(461, 15)
6 /461 in the top 1% in 1955,1965
(4868, 15)
50 /4868 in the top 1% in 1960,1970
(14321, 15)
168 /14321 in the top 1% in 1965,1975
(26267, 15)
264 /26267 in the top 1% in 1970,1980
(36582, 15)
370 /36582 in the top 1% in 1975,1985
(47260, 15)
482 /47260 in the top 1% in 1980,1990


In [5]:
toptops = pd.concat(toptops)

In [8]:
toptops.shape

(1351, 15)

In [9]:
save_variable("%s.top1" % database_name, toptops)